In [1]:
import os
os.environ["FLASK_ENV"]='development'

from mdf_connect_server.automate.globus_automate_flow import GlobusAutomateFlow
import mdf_toolbox
import json
import time

In [2]:
native_app_id = "417301b1-5101-456a-8a27-423e71a2ae26"  # Premade native app ID
automate = GlobusAutomateFlow(native_app_id)
automate.read_flow("mdf_flow_info.json")

print(automate)

Globus Automate Flow: id=33af393f-4c5d-4031-ba2d-0c898de7c4fd, scope=https://auth.globus.org/scopes/33af393f-4c5d-4031-ba2d-0c898de7c4fd/flow_33af393f_4c5d_4031_ba2d_0c898de7c4fd_user


In [3]:
# Generate input credentials
# The feedstock auth header is shorter-lived, so it's not set in the Flow defintition.
# However, the header must be from an identity with access to the Xtract output location (currently Petrel)
petrel_auth = mdf_toolbox.login(services=["petrel"], make_clients=False)["petrel"]
feedstock_auth_header = {}
petrel_auth.set_authorization_header(feedstock_auth_header)
feedstock_auth_header = feedstock_auth_header["Authorization"]

# The RunAs token is used to Transfer data from the user as the user's identity.
# It will be given by the MDF user (by logging in with MDF).
# NOTE: Currently, RunAs is not used on the Transfer.
run_as_auth = automate.get_runas_auth()
run_as_token = run_as_auth.refresh_token

In [4]:
# Get test input for Flow
with open("mdf_flow_input.json") as f:
    flow_input = json.load(f)

# Add credentials to input
flow_input["_private_feedstock_auth_header"] = feedstock_auth_header
flow_input["_tokens"] = {"MDFUser": run_as_token}

# Create unique source_id for submission (source_id must be unique except for updates)
source_id = "mdf_flow_test_" + str(int(time.time())) + "_v1.1"
flow_input["source_id"] = source_id
source_id

'mdf_flow_test_1606930993_v1.1'

In [5]:
flow = automate.run_flow(flow_input)

In [6]:
print(json.dumps(flow.get_status(), indent=4, sort_keys=True))

{
    "action_id": "a141115e-f938-4111-8b6f-769af74c9f94",
    "completion_time": "None",
    "created_by": "urn:globus:auth:identity:8dd693cf-f1a0-4ee4-8079-b69705523e6b",
    "details": {
        "code": "ActionStarted",
        "description": "State UserTransfer of type Action started",
        "details": {
            "input": {
                "action_inputs": [
                    {
                        "destination_endpoint_id": "e38ee745-6d04-11e5-ba46-22000b92c6ec",
                        "label": "MDF Flow Test Transfer1",
                        "source_endpoint_id": "e38ee745-6d04-11e5-ba46-22000b92c6ec",
                        "transfer_items": [
                            {
                                "destination_path": "/MDF/mdf_connect/test_files/deleteme/data/test123/",
                                "recursive": true,
                                "source_path": "/MDF/mdf_connect/test_files/canonical_datasets/dft/"
                            }
         

In [11]:
flow.get_error_msgs()

{'code': 'ActionFailed', 'details': {'cause': '{\'action_id\': \'46c3dee4-87de-448e-9460-cc8109c02ecc\', \'creator_id\': \'urn:globus:auth:identity:8dd693cf-f1a0-4ee4-8079-b69705523e6b\', \'details\': {\'message\': \'Action started\'}, \'errors\': ["error ingesting 46c3dee4-87de-448e-9460-cc8109c02ecc/mock_feedstock.json: (403, \'Forbidden.Generic\', \'ingest request denied by service\')"], \'manage_by\': [\'urn:globus:auth:identity:256d4736-e319-4ccc-b7d2-88f0cc2f8357\', \'urn:globus:auth:identity:4d40f211-15ac-4106-b8f5-e73dc093b5d5\', \'urn:globus:auth:identity:8dd693cf-f1a0-4ee4-8079-b69705523e6b\'], \'monitor_by\': [\'urn:globus:auth:identity:256d4736-e319-4ccc-b7d2-88f0cc2f8357\', \'urn:globus:auth:identity:4d40f211-15ac-4106-b8f5-e73dc093b5d5\', \'urn:globus:auth:identity:8dd693cf-f1a0-4ee4-8079-b69705523e6b\'], \'release_after\': \'P30D\', \'request_id\': \'flows_req_e60d29cd-c0c7-46c2-a234-12b04dea6f18\', \'status\': \'FAILED\'}'}, 'time': '2020-12-02T17:43:56.497000+00:00'}
f

[["error ingesting 46c3dee4-87de-448e-9460-cc8109c02ecc/mock_feedstock.json: (403, 'Forbidden.Generic', 'ingest request denied by service')"]]

In [8]:
flow.action_id

'a141115e-f938-4111-8b6f-769af74c9f94'

In [9]:
automate

In [10]:
print(automate)

Globus Automate Flow: id=33af393f-4c5d-4031-ba2d-0c898de7c4fd, scope=https://auth.globus.org/scopes/33af393f-4c5d-4031-ba2d-0c898de7c4fd/flow_33af393f_4c5d_4031_ba2d_0c898de7c4fd_user
